In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# replace with your actual filename
with open("/Users/franri/Downloads/Google.html", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

#print(soup)

industries = [
    "Automotive & Logistics",
    "Business & Professional Services",
    "Financial Services",
    "Healthcare & Life Sciences",
    "Hospitality & Travel",
    "Manufacturing, Industrial & Electronics",
    "Media, Marketing & Gaming",
    "Public Sector & Nonprofits",
    "Retail",
    "Technology",
    "Telecommunications"
]

rows = []

for industry in industries:
    # 1) find the <p> tag carrying the industry name
    p = soup.find("p", string=lambda t: t and t.strip() == industry)
    if not p:
        print(f"❌ Industry not found in <p>: {industry!r}")
        continue
    print(f"\n🔎 Found industry <p>: {industry!r}")

    # 2) step up to its wrapping <section>
    sec_ind = p.find_parent("section", class_="QzPuud")
    if not sec_ind:
        print("   ⚠️  No parent <section> for this <p>")
        continue

    # 3) find the *next* <section> sibling that has at least one <h3>
    types_sec = sec_ind.find_next_sibling()
    while types_sec and (types_sec.name != "section" or not types_sec.find("h3")):
        types_sec = types_sec.find_next_sibling()
    if not types_sec:
        print("   ⚠️  No following <section> with <h3> found")
        continue
    print("   ✅  Types section found with classes:", types_sec.get("class"))

    # 4) inside that, each <h3> is a Type
    for h3 in types_sec.find_all("h3"):
        type_name = h3.get_text(strip=True)
        print(f"    • Type: {type_name!r}")

        # 5) its companies live in the very next <ul>
        ul = h3.find_next_sibling("ul")
        if not ul:
            print("       – no <ul> after this <h3>, skipping")
            continue

        # 6) each <li> → one company + description
        for li in ul.find_all("li"):
            strong = li.find("strong")
            if not strong:
                continue
            raw = strong.get_text(strip=True)
            company = raw.lstrip("*")
            full = li.get_text(separator=" ", strip=True)
            # strip the company name off the front of the description
            desc = full[len(raw):].strip() if full.startswith(raw) else full
            desc = desc.lstrip("–—:;., ")
            print(f"       - {company!r}: {desc[:60]!r}")

            rows.append({
                "Industry":    industry,
                "Type":        type_name,
                "Company":     company,
                "Description": desc
            })

print(f"\n🏁 Total rows collected: {len(rows)}")
df = pd.DataFrame(rows)
print(df)


🔎 Found industry <p>: 'Automotive & Logistics'
   ✅  Types section found with classes: ['Wy08Ac', 'nRhiJb-qJTHM-OWXEXe-hJDwNd', 'nRhiJb-DbgRPb-II5mzb-cGMI2b', 'gGhBwf']
    • Type: 'Customer Agents'
       - 'Continental': "is using Google's data and AI technologies to develop automo"
       - 'General Motors': '’ OnStar has been augmented with new AI features, including '
       - 'MercedesBenz': '* MercedesBenz is providing conversational search and naviga'
       - 'Mercedes Benz': 'is infusing e-commerce capabilities into its online storefro'
       - 'PODS': 'worked with the advertising agency Tombras to create the “Wo'
       - 'UPS Capital': 'launched DeliveryDefense Address Confidence, which uses mach'
       - 'Volkswagen of America': 'built a virtual assistant in the myVW app, where drivers can'
    • Type: 'Employee Agents'
       - '704 Apps': 'creates applications serving the last-mile transportation se'
       - 'Oxa': 'a developer of software for autonomous vehicles, us

In [ ]:
df["Description"] = df["Company"] + " " + df["Description"]
df.head(15)

,Industry,Type,Company,Description
0,Automotive & Logistics,Customer Agents,Continental,Continental is using Google's data and AI tech...
1,Automotive & Logistics,Customer Agents,General Motors,General Motors ’ OnStar has been augmented wit...
2,Automotive & Logistics,Customer Agents,MercedesBenz,MercedesBenz * MercedesBenz is providing conve...
3,Automotive & Logistics,Customer Agents,Mercedes Benz,Mercedes Benz is infusing e-commerce capabilit...
4,Automotive & Logistics,Customer Agents,PODS,PODS worked with the advertising agency Tombra...
5,Automotive & Logistics,Customer Agents,UPS Capital,UPS Capital launched DeliveryDefense Address C...
6,Automotive & Logistics,Customer Agents,Volkswagen of America,Volkswagen of America built a virtual assistan...
7,Automotive & Logistics,Employee Agents,704 Apps,704 Apps creates applications serving the last...
8,Automotive & Logistics,Employee Agents,Oxa,Oxa a developer of software for autonomous veh...
9,Automotive & Logistics,Employee Agents,Rivian,Rivian * Rivian uses Google Workspace with Gem...


In [43]:
df.to_csv("/Users/franri/Downloads/output.csv", index=False)